In [1]:
import tensorflow as tf
from tensorflow.python.ops import rnn, rnn_cell
import numpy as np

In [2]:
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
'''
To classify images using a reccurent neural network, we consider every image
row as a sequence of pixels. Because MNIST image shape is 28*28px, we will then
handle 28 sequences of 28 steps for every sample.
'''

'\nTo classify images using a reccurent neural network, we consider every image\nrow as a sequence of pixels. Because MNIST image shape is 28*28px, we will then\nhandle 28 sequences of 28 steps for every sample.\n'

In [4]:
# Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 2
display_step = 10

# Network Parameters
n_input = 28 # MNIST data input (img shape: 28*28)
n_steps = 28 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [5]:
# Permuting batch_size and n_steps
x_transpose = tf.transpose(x, [1, 0, 2])
# Reshaping to (n_steps*batch_size, n_input)
x_reshape = tf.reshape(x_transpose, [-1, n_input])
# Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
x_nstep = tf.split(0, n_steps, x_reshape)
# Define a lstm cell with tensorflow
lstm_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)
# Get lstm cell output
outputs, states = rnn.rnn(lstm_cell, x_nstep, dtype=tf.float32)
#the output score
pred = tf.matmul(outputs[-1], weights['out']) + biases['out']

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.initialize_all_variables()

In [16]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    batch_x = batch_x.reshape((batch_size, n_steps, n_input))
    state = sess.run(states, feed_dict={x: batch_x})
    output = sess.run(outputs, feed_dict={x: batch_x})

    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc)
        step += 1
        
    print "Optimization Finished!"
    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print "Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label})


batch_x.shape
(2, 28, 28)
x.shape
(2, 28, 28)
x_transpose.shape
(28, 2, 28)
x_reshape.shape
(56, 28)
output
0
[[  1.37320414e-01   7.82275349e-02   5.95216826e-02   1.50546461e-01
    3.57302696e-01  -3.20153415e-01   2.80332327e-01  -5.14831245e-02
    1.47666335e-01   1.38003841e-01  -3.64056565e-02   1.30510060e-02
    3.13133329e-01  -1.28339101e-02   2.79234946e-01   9.80778597e-03
   -1.79315716e-01   4.74456511e-02   2.72651494e-01  -2.54500210e-01
    5.32120839e-02  -1.33881420e-02   4.74193037e-01   6.32767528e-02
   -1.18695781e-01   1.72720075e-01   1.43118799e-01  -4.67648447e-01
    2.07874179e-01  -3.34274232e-01  -1.49666026e-01  -1.56564996e-01
   -5.00640236e-02   2.18375660e-02  -4.48620245e-02   9.53665301e-02
   -8.08745101e-02  -1.06309792e-02  -2.53331095e-01  -3.20565939e-01
   -6.43034130e-02   1.83173910e-01  -1.10994332e-01  -5.83995357e-02
    1.10582441e-01  -2.96124518e-01  -3.23962927e-01   3.64838801e-02
    1.14526436e-01  -4.79082018e-02  -1.68261737e-